# Face Alignment - Model Testing

- Add the project's root directory (two levels up) to the Python path so the modules can be imported, even if they arent in the current working directory:

In [ ]:
import sys
import os

project_root = os.path.abspath(os.path.join('..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

- Import the required libraries and modules, as well as our utility functions:

In [ ]:
import numpy as np
import joblib
import matplotlib.pyplot as plt
import pandas as pd

from src.utils import load_config, get_project_root, save_as_npz

- Load the config using the utility function. Get paths to relevant folders/files needed to save and retrieve files:

In [ ]:
config = load_config()

test_path = config['data']['task2']['processed']['test']

test_data_path = os.path.join(get_project_root(), test_path, 'processed_face_alignment_test_images_features.npz')
model_path = os.path.join(get_project_root(), config['data']['task2']['models'], 'best_model.pkl')
pca_path = os.path.join(get_project_root(), config['data']['task2']['models'], 'pca_transform.pkl')
output_path = os.path.join(get_project_root(), config['data']['task2']['results'])

results_output_path = os.path.join(get_project_root(), output_path.replace('/', os.sep))

- Load the trained model from the specified filepath. Load the test data that we want to produce the results on. For consistency, I also decided to save and load the same PCA:

In [ ]:
best_model = joblib.load(model_path)
test_data = np.load(test_data_path, allow_pickle=True)
pca = joblib.load(pca_path)

- References to the features and images of the test data. Our images are provided but the features were extracted during an earlier stage:

In [ ]:
X_test = test_data['features']
images = test_data['images']

- Apply the PCA to the test data, and make predictions using the trained model. Reshape the predictions into a 3D array to match the format:

In [ ]:
X_test_pca = pca.transform(X_test)

y_pred = best_model.predict(X_test_pca)
y_pred_reshaped = y_pred.reshape((y_pred.shape[0], -1, 2)) 

- Produce some samples of our predicted landmarks on the provided images in the test set. This will give us a good idea as to whether our model is working as effectively as desired:

In [ ]:
for i in range(3):
    plt.imshow(images[i], cmap='gray')
    plt.scatter(y_pred_reshaped[i][:, 0], y_pred_reshaped[i][:, 1], c='r', s=10, label='Prediction')
    plt.title(f"Predicted Landmarks - Sample {i}")
    plt.axis('off')
    plt.legend()
    plt.show()

- Save the data as a CSV file. This CSV file will contain the results of our predictions.

In [ ]:
image_flattened = images.reshape(images.shape[0], -1)
points_flattened = y_pred_reshaped.reshape(y_pred_reshaped.shape[0], -1)

In [ ]:
df_images = pd.DataFrame(image_flattened)
df_points = pd.DataFrame(points_flattened)

df_combined = pd.concat([df_images, df_points], axis=1)
df_combined.head()
#df_combined.columns = [f"Pixel_{i}" for i in range(df_images.shape[1])] + [f"Landmark_{i//2}_{'X' if i%2 == 0 else 'Y'}" for i in range(df_points.shape[1])]

In [ ]:
save_as_csv(df_combined, results_output_path, "results.csv")